<a href="https://colab.research.google.com/github/atanuGuin/NLP_Coursework/blob/master/BERT_of_Email_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Necessary Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os, email
import pandas as pd
import glob
from tqdm.notebook import tqdm
import numpy as np
import re
import string
import time
import nltk
import random

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 14.9 MB/s 
     |████████████████████████████████| 182 kB 74.6 MB/s 
     |████████████████████████████████| 7.6 MB 70.9 MB/s 


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 14.6 MB/s 
     |████████████████████████████████| 212 kB 77.9 MB/s 
     |████████████████████████████████| 132 kB 77.8 MB/s 
     |████████████████████████████████| 127 kB 80.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
from sklearn.preprocessing import LabelEncoder

In [ ]:
import torch
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification

# Define the data path

In [ ]:
PATH = '/content/drive/MyDrive/Demo/enron_with_categories'


#Loading the Data based on 6 categories

In [ ]:
category_dict = {1: "Company Business, Strategy, etc." ,
                 2: "Purely Personal",
                 3: "Personal but in professional context",
                 4: "Logistic Arrangements",
                 5: "Employment arrangements",
                 6: "Document editing/checking"}

files = []
text = []
category = []


for path in tqdm(glob.glob(PATH + "/[1-6]/*.txt")):
    batch,filename = path.split("/")[-2:]
    contents = open(path, "r").read()
    files.append(filename)
    text.append(contents)
    category.append(category_dict[int(batch)])



  0%|          | 0/1663 [00:00<?, ?it/s]

In [ ]:
emails_df = pd.DataFrame(list(zip(files,text, category)),columns = ["file", "message", "category"])

In [ ]:
emails_df.to_csv("emails.csv", index=False)

In [ ]:
emails_df = pd.read_csv("emails.csv" )

In [ ]:
emails_df

,file,message,category
0,175741.txt,Message-ID: <31425995.1075847622214.JavaMail.e...,"Company Business, Strategy, etc."
1,55205.txt,Message-ID: <18699857.1075856630781.JavaMail.e...,"Company Business, Strategy, etc."
2,120674.txt,Message-ID: <20878896.1075843391140.JavaMail.e...,"Company Business, Strategy, etc."
3,173929.txt,Message-ID: <10985446.1075846163264.JavaMail.e...,"Company Business, Strategy, etc."
4,173918.txt,Message-ID: <966711.1075846162973.JavaMail.eva...,"Company Business, Strategy, etc."
...,...,...,...
1658,229326.txt,Message-ID: <26808496.1075853126387.JavaMail.e...,Personal but in professional context
1659,12176.txt,Message-ID: <31386690.1075860837352.JavaMail.e...,Personal but in professional context
1660,114495.txt,Message-ID: <24509319.1075842948701.JavaMail.e...,Personal but in professional context
1661,12174.txt,Message-ID: <28168211.1075860837271.JavaMail.e...,Personal but in professional context


#Preprocessing the Data

##Step 1: Extract different parts an email from the text files like Sender, Receiver, Subject, Content, Message type etc. to create a dataframe

In [ ]:
## Helper functions
def get_text_from_email(msg, max_word_len=30):
    '''To get the content from email objects'''
    parts = []
    for part in msg.walk():
        if part.get_content_type() == 'text/plain':
            payload = part.get_payload()
            payload = ' '.join(filter(lambda x: len(x) < max_word_len,  payload.split()))
            parts.append( payload )
    return ''.join(parts)

def split_email_addresses(line):
    '''To separate multiple email addresses'''
    if line:
        addrs = line.split(',')
        addrs = frozenset(map(lambda x: x.strip(), addrs))
    else:
        addrs = None
    return addrs

In [ ]:
# Parse the emails into a list email objects
messages = list(map(email.message_from_string, emails_df['message']))

# Get fields from parsed email objects
keys = messages[0].keys()
for key in keys:
    emails_df[key] = [doc[key] for doc in messages]
# Parse content from emails
emails_df['content'] = list(map(get_text_from_email, messages))
# Split multiple email addresses
emails_df['From'] = emails_df['From'].map(split_email_addresses)
emails_df['To'] = emails_df['To'].map(split_email_addresses)

# cleanup
del messages
emails_df.drop('message', axis=1, inplace=True)

emails_df.head()

,file,category,Message-ID,Date,From,To,Subject,Cc,Mime-Version,Content-Type,Content-Transfer-Encoding,Bcc,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,content
0,175741.txt,"Company Business, Strategy, etc.",<31425995.1075847622214.JavaMail.evans@thyme>,"Fri, 9 Mar 2001 09:28:00 -0800 (PST)",(steven.kean@enron.com),(pamela.lesh@enron.com),Re: Several matters,"joe.hartsoe@enron.com, james.derrick@enron.com...",1.0,text/plain; charset=us-ascii,7bit,"joe.hartsoe@enron.com, james.derrick@enron.com...",Steven J Kean,Pamela Lesh,"Joe Hartsoe, James Derrick, Linda Robertson",,\Steven_Kean_June2001_1\Notes Folders\All docu...,KEAN-S,skean.nsf,Thanks for the message. It's good to hear from...
1,55205.txt,"Company Business, Strategy, etc.",<18699857.1075856630781.JavaMail.evans@thyme>,"Thu, 1 Mar 2001 06:29:00 -0800 (PST)",(hillh@stanford.edu),(vkamins@enron.com),possible RTP conference,None,1.0,text/plain; charset=us-ascii,7bit,None,Hill Huntington <hillh@stanford.edu>,vkamins@enron.com,,,\Vincent_Kaminski_Jun2001_5\Notes Folders\Stan...,Kaminski-V,vkamins.nsf,Dear Mr. Kaminski: Thank you for talking with ...
2,120674.txt,"Company Business, Strategy, etc.",<20878896.1075843391140.JavaMail.evans@thyme>,"Wed, 14 Mar 2001 05:47:00 -0800 (PST)",(alan.comnes@enron.com),(susan.mara@enron.com),Re: You Can Put A Stake in Their Hearts but Th...,"richard.shapiro@enron.com, james.steffes@enron...",1.0,text/plain; charset=ANSI_X3.4-1968,quoted-printable,"richard.shapiro@enron.com, james.steffes@enron...",Alan Comnes,Susan J Mara,"Richard Shapiro, James D Steffes, Joe Hartsoe,...",,\Jeff_Dasovich_June2001\Notes Folders\All docu...,DASOVICH-J,jdasovic.nsf,Here's the proposal to form the Team. Note tha...
3,173929.txt,"Company Business, Strategy, etc.",<10985446.1075846163264.JavaMail.evans@thyme>,"Wed, 2 Aug 2000 11:33:00 -0700 (PDT)",(steven.kean@enron.com),(scott.tholan@enron.com),Enron projects with OPIC financing,None,1.0,text/plain; charset=us-ascii,7bit,None,Steven J Kean,Scott Tholan,,,\Steven_Kean_Dec2000_1\Notes Folders\All docum...,KEAN-S,skean.nsf,OPIC information ---------------------- Forwar...
4,173918.txt,"Company Business, Strategy, etc.",<966711.1075846162973.JavaMail.evans@thyme>,"Wed, 2 Aug 2000 08:05:00 -0700 (PDT)",(steven.kean@enron.com),(jon.thomsen@enron.com),Ken Lay's Meeting with Fox,"ken.rice@enron.com, ricardo.charvel@enron.com,...",1.0,text/plain; charset=us-ascii,7bit,"ken.rice@enron.com, ricardo.charvel@enron.com,...",Steven J Kean,Jon Thomsen,"Ken Rice, Ricardo Charvel, Rosalee Fleming",,\Steven_Kean_Dec2000_1\Notes Folders\All docum...,KEAN-S,skean.nsf,Jon -- Ricardo Charvel is pulling together the...


##Step 2: Checking the dataframe for unique values and dropping irrelevant features for email classification task


In [ ]:
print('shape of the dataframe:', emails_df.shape)
# Find number of unique values in each columns
for col in emails_df.columns:
    print(col, emails_df[col].nunique())

shape of the dataframe: (1663, 20)
file 1663
category 6
Message-ID 1663
Date 1571
From 172
To 696
Subject 1242
Cc 288
Mime-Version 1
Content-Type 2
Content-Transfer-Encoding 2
Bcc 288
X-From 202
X-To 842
X-cc 319
X-bcc 1
X-Folder 180
X-Origin 76
X-FileName 93
content 1554


## Step 3: Only Subject and Email content have been taken for the classification task

In [ ]:
emails_df.columns

Index(['file', 'category', 'Message-ID', 'Date', 'From', 'To', 'Subject', 'Cc',
       'Mime-Version', 'Content-Type', 'Content-Transfer-Encoding', 'Bcc',
       'X-From', 'X-To', 'X-cc', 'X-bcc', 'X-Folder', 'X-Origin', 'X-FileName',
       'content'],
      dtype='object')

In [ ]:
emails_df['text'] = emails_df['Subject'] + " " + emails_df['content']

In [ ]:
# drop the columns 'subject' and 'body'
emails_df.drop(['file', 'Message-ID', 'Date', 'From', 'To', 'Subject',
       'Mime-Version', 'Content-Type', 'Content-Transfer-Encoding',
       'X-From', 'X-To', 'X-cc', 'X-bcc', 'X-Folder', 'X-Origin', 'X-FileName', 'content'], axis=1, inplace=True)

In [ ]:
emails_df

,category,Cc,Bcc,text
0,"Company Business, Strategy, etc.","joe.hartsoe@enron.com, james.derrick@enron.com...","joe.hartsoe@enron.com, james.derrick@enron.com...",Re: Several matters Thanks for the message. It...
1,"Company Business, Strategy, etc.",None,None,possible RTP conference Dear Mr. Kaminski: Tha...
2,"Company Business, Strategy, etc.","richard.shapiro@enron.com, james.steffes@enron...","richard.shapiro@enron.com, james.steffes@enron...",Re: You Can Put A Stake in Their Hearts but Th...
3,"Company Business, Strategy, etc.",None,None,Enron projects with OPIC financing OPIC inform...
4,"Company Business, Strategy, etc.","ken.rice@enron.com, ricardo.charvel@enron.com,...","ken.rice@enron.com, ricardo.charvel@enron.com,...",Ken Lay's Meeting with Fox Jon -- Ricardo Char...
...,...,...,...,...
1658,Personal but in professional context,a..hope@enron.com,a..hope@enron.com,RE: Confidential Concern Valeria is looking in...
1659,Personal but in professional context,None,None,A Supportive Note Dear Mr. Lay: I'm writing th...
1660,Personal but in professional context,None,None,[Second Delivery: WPTF Friday Amen Burrito] So...
1661,Personal but in professional context,None,None,Thank you all very much for your support - NOT...


##Step 4: Preprocessing the context by converting it to lowercase, removing extra lines and white spaces, removing punctuations and stopwords

In [ ]:
def preprocess(x):
    # lowercasing all the words
    x = x.lower()
    
    # remove extra new lines
    x = re.sub(r'\n+', ' ', x)
    
    # removing (replacing with empty spaces actually) all the punctuations
    x = re.sub("["+string.punctuation+"]", " ", x)
    
    # remove extra white spaces
    x = re.sub(r'\s+', ' ', x)
    
    return x

In [ ]:
start = time.time()
emails_df.loc[:,'text'] = emails_df.loc[:, 'text'].map(preprocess)

# remove stopwords
# emails_df.loc[:, 'text'] = emails_df.loc[:, 'text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop]))
end = time.time()
print("Execution time (sec): ",(end - start))

Execution time (sec):  0.7746226787567139


In [ ]:
emails_df

,category,Cc,Bcc,text
0,"Company Business, Strategy, etc.","joe.hartsoe@enron.com, james.derrick@enron.com...","joe.hartsoe@enron.com, james.derrick@enron.com...",re several matters thanks for the message it s...
1,"Company Business, Strategy, etc.",None,None,possible rtp conference dear mr kaminski thank...
2,"Company Business, Strategy, etc.","richard.shapiro@enron.com, james.steffes@enron...","richard.shapiro@enron.com, james.steffes@enron...",re you can put a stake in their hearts but the...
3,"Company Business, Strategy, etc.",None,None,enron projects with opic financing opic inform...
4,"Company Business, Strategy, etc.","ken.rice@enron.com, ricardo.charvel@enron.com,...","ken.rice@enron.com, ricardo.charvel@enron.com,...",ken lay s meeting with fox jon ricardo charvel...
...,...,...,...,...
1658,Personal but in professional context,a..hope@enron.com,a..hope@enron.com,re confidential concern valeria is looking int...
1659,Personal but in professional context,None,None,a supportive note dear mr lay i m writing this...
1660,Personal but in professional context,None,None,second delivery wptf friday amen burrito sorr...
1661,Personal but in professional context,None,None,thank you all very much for your support not i...


##Step 5: Encoding the 6 labels for as target

In [ ]:
possible_labels = emails_df.category.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'Company Business, Strategy, etc.': 0,
 'Employment arrangements': 1,
 'Purely Personal': 2,
 'Logistic Arrangements': 3,
 'Document editing/checking': 4,
 'Personal but in professional context': 5}

In [ ]:
def label_encoder(data):
    class_le = LabelEncoder()
    # apply label encoder on the 'Category' column
    y = class_le.fit_transform(data)
    return y

In [ ]:
emails_df['label']= pd.DataFrame(label_encoder(emails_df['category']))

In [ ]:
emails_df.drop("category", axis = 1)

,Cc,Bcc,text,label
0,"joe.hartsoe@enron.com, james.derrick@enron.com...","joe.hartsoe@enron.com, james.derrick@enron.com...",re several matters thanks for the message it s...,0
1,None,None,possible rtp conference dear mr kaminski thank...,0
2,"richard.shapiro@enron.com, james.steffes@enron...","richard.shapiro@enron.com, james.steffes@enron...",re you can put a stake in their hearts but the...,0
3,None,None,enron projects with opic financing opic inform...,0
4,"ken.rice@enron.com, ricardo.charvel@enron.com,...","ken.rice@enron.com, ricardo.charvel@enron.com,...",ken lay s meeting with fox jon ricardo charvel...,0
...,...,...,...,...
1658,a..hope@enron.com,a..hope@enron.com,re confidential concern valeria is looking int...,4
1659,None,None,a supportive note dear mr lay i m writing this...,4
1660,None,None,second delivery wptf friday amen burrito sorr...,4
1661,None,None,thank you all very much for your support not i...,4


## Train Test split of the Dataset with 80% train and 20% test set

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(emails_df.index.values, 
                                                  emails_df.label.values, 
                                                  test_size=0.2, 
                                                  random_state=42, 
                                                  stratify=emails_df.label.values)

emails_df['data_type'] = ['not_set']*emails_df.shape[0]
emails_df.loc[X_train, 'data_type'] = 'train'
emails_df.loc[X_val,  'data_type'] = 'val'

emails_df.groupby(['category', 'label', 'data_type']).count()

Cc  Bcc  text
category                             label data_type                
Company Business, Strategy, etc.     0     train      201  201   667
                                           val         57   57   167
Document editing/checking            1     train       32   32   114
                                           val          8    8    29
Employment arrangements              2     train       17   17    59
                                           val          4    4    15
Logistic Arrangements                3     train       64   64   381
                                           val         13   13    95
Personal but in professional context 4     train        8    8    80
                                           val          2    2    20
Purely Personal                      5     train        4    4    29
                                           val          1    1     7

##Tokenize the corpus and Encoding the tokens the Train and Test Split separately using BERT Tokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus(
    emails_df[emails_df.data_type=='train'].text.values, 
    add_special_tokens=False, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt',
    truncation=True
)

encoded_data_val = tokenizer.batch_encode_plus(
    emails_df[emails_df.data_type=='val'].text.values, 
    add_special_tokens=False, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt',
    truncation=True
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(emails_df[emails_df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(emails_df[emails_df.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
len(dataset_train), len(dataset_val)

(1330, 333)

## Creating Data Loader for the Train and Test Data

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 8

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

##Creating Functions for Evaluation Metrics

In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    accr = 0
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        accr += len(y_preds[y_preds==label])
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')
    print(f'Average Accuracy across all classes: {accr/len(labels_flat)}\n')


## Seeding with a constant value for creating reproducable results

In [ ]:
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

#Download the BERT Pretrained model for classification of 6 labels

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)


Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

#Define the epochs, optimizer and learning rate scheduler

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 10

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

#Move the Model to Run on Cuda 

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


#Create Evaluator to get validation metrics

In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    


#Train the BERT Model

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/167 [00:00<?, ?it/s]


Epoch 1
Training loss: 1.3212863299661053
Validation loss: 1.1986299121663684
F1 Score (Weighted): 0.5287498892807068


Epoch 2:   0%|          | 0/167 [00:00<?, ?it/s]


Epoch 2
Training loss: 1.0968856854353122
Validation loss: 1.0614599986445337
F1 Score (Weighted): 0.5872652253825092


...


Epoch 8
Training loss: 0.33650010297755284
Validation loss: 1.0681191515177488
F1 Score (Weighted): 0.6749877123526483


Epoch 9:   0%|          | 0/167 [00:00<?, ?it/s]


Epoch 9
Training loss: 0.2934532065368341
Validation loss: 1.1075689203238912
F1 Score (Weighted): 0.6750528186416392


Epoch 10:   0%|          | 0/167 [00:00<?, ?it/s]


Epoch 10
Training loss: 0.26984663149286175
Validation loss: 1.1067564443995555
F1 Score (Weighted): 0.6711471078274357


#Find the prediction on Test Set

In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)


# Accuracy on Test Set

In [ ]:
accuracy_per_class(predictions, true_vals)

Class: Company Business, Strategy, etc.
Accuracy: 141/167

Class: Employment arrangements
Accuracy: 13/29

Class: Purely Personal
Accuracy: 3/15

Class: Logistic Arrangements
Accuracy: 63/95

Class: Document editing/checking
Accuracy: 3/20

Class: Personal but in professional context
Accuracy: 1/7

Average Accuracy across all classes: 0.6726726726726727



Result Analysis:
As we can see from the result, Bert Transformer is providing us better results in multiclass classifications